In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import json
from services.evaluation import Evaluator, Run
from retrieval.neural.modules.encoders import MaxPoolEncoder, MeanPoolEncoder
from retrieval.neural.modules.scorers import LinearLogisticRegression
from services.index import Index
import unidecode
from tqdm import tqdm
import torch
import pytrec_eval
from services import sql
import sqlite3
import pickle
import pandas as pd

In [4]:
encoder = MaxPoolEncoder(50)

In [5]:
encoder.embed(torch.tensor(1).view(1,1))

tensor([[[ 0.4180,  0.2497, -0.4124,  0.1217,  0.3453, -0.0445, -0.4969,
          -0.1786, -0.0007, -0.6566,  0.2784, -0.1477, -0.5568,  0.1466,
          -0.0095,  0.0117,  0.1020, -0.1279, -0.8443, -0.1218, -0.0168,
          -0.3328, -0.1552, -0.2313, -0.1918, -1.8823, -0.7675,  0.0991,
          -0.4212, -0.1953,  4.0071, -0.1859, -0.5229, -0.3168,  0.0006,
           0.0074,  0.1778, -0.1590,  0.0120, -0.0542, -0.2987, -0.1575,
          -0.3476, -0.0456, -0.4425,  0.1878,  0.0028, -0.1841, -0.1151,
          -0.7858]]])

In [9]:
query = encoder(torch.tensor([1,2]).view(1,2))

In [10]:
doc = encoder(torch.tensor([1,2]).view(1,2))

In [11]:
scorer(query, doc)

tensor([[1.0000]], grad_fn=<ViewBackward>)

In [10]:
with open('./data/index/token2id.tar', 'rb') as file:
    t2i = pickle.load(file)
with open('./data/index/id2tf.tar', 'rb') as file:
    i2tf = pickle.load(file)

In [13]:
for t in list(t2i.keys())[0:100]:
    print(f'{t}\t\t{t2i[t]}\t{i2tf[t2i[t]]}')

the		1	33518850
of		2	17423417
in		3	16323243
0eos0		4	14893419
and		5	13957525
a		6	11768802
is		7	8258719
to		8	7602930
was		9	7171948
0eop0		10	6569478
as		11	3922432
he		12	3782949
by		13	3739468
for		14	3717419
on		15	3670719
it		16	3128950
at		17	2992988
with		18	2889002
from		19	2767686
an		20	2383033
his		21	2250277
that		22	1599742
are		23	1288875
also		24	1270719
born		25	1267274
which		26	1249189
first		27	1228057
has		28	1210251
s		29	1176048
or		30	1117400
its		31	1051223
who		32	1003060
one		33	980661
were		34	957796
this		35	934600
new		36	918858
she		37	899686
after		38	811843
school		39	775686
two		40	770519
known		41	745881
may		42	736572
be		43	720539
her		44	706083
american		45	704928
university		46	701714
district		47	691992
their		48	677463
had		49	674220
county		50	663662
united		51	658472
south		52	636136
but		53	624799
national		54	624705
1		55	616609
during		56	585705
they		57	582301
been		58	578535
film		59	576031
city		60	573364
state		61	565450
located		62	

In [3]:
index = Index('tfidf')

INFO:root:[2018-12-06 03:34:04.823619]	[79721]	Loading index ./data/index/indri with tfidf query environment.
INFO:root:[2018-12-06 03:34:09.403098]	[79721]	Not loading stopwords.
INFO:root:[2018-12-06 03:34:17.130822]	[79721]	Loaded index in 0:00:12.306049.


In [5]:
%timeit index.unigram_query('What is anarchism?', 1000)

981 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
def fn():
    with sqlite3.connect('./data/term_retrievals/uni_tfidf.dummy/retrievals.sqlite') as db:
        (q_id, target_titles, doc_int_ids) = db.cursor() \
            .execute(sql.get_question_by_qid(), ('5a7a52745542996c55b2dd4f',)) \
            .fetchone()
        return pickle.loads(doc_int_ids)[:1000]

In [10]:
%timeit fn()

859 µs ± 33.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [3]:
df = pd.read_pickle('./data/candidates.tfidf.tar')

In [4]:
df.head()

,question_id,document_id,target
0,5a7a06935542990198eaf050,1662997,1
1,5a7a06935542990198eaf050,4820929,0
2,5a7a06935542990198eaf050,1906799,0
3,5a7a06935542990198eaf050,3595436,0
4,5a7a06935542990198eaf050,43568,0


In [31]:
run = Run()

In [32]:
run.add_ranking("a", {"a1": 1})
run.add_ranking("b", {"b1": 1, "b2": 2})
run.update_ranking("d", "d1", 3)

-1


In [10]:
run.update_rankings({"c": {"c1": 3}, "b": {"b3": 3}})

In [27]:
run

{'a': {'a1': 1}, 'b': {'b1': 1, 'b2': 2}, 'd': {}}

-1


KeyError: 'd1'

In [7]:
run

{'a': {'a1': 1}, 'b': {'b1': 1, 'b2': 2}, 'd': {'d1': 3}}

In [19]:
# with open() as file:
df = pd.read_pickle('./data/candidates/tfidf.train.dummy.gzip', compression='gzip')

In [20]:
df[0:1000]

,question_id,document_id,question,document,target
0,5a7a06935542990198eaf050,4631905,"[26, 650, 9, 327, 27, 1504, 29, 650, 30, 27, 1...","[22909, 650, 22909, 650, 4431, 27629, 9, 20, 4...",1
1,5a7a06935542990198eaf050,1662997,"[26, 650, 9, 327, 27, 1504, 29, 650, 30, 27, 1...","[27, 14, 733, 27, 14, 733, 7, 6, 8800, 650, 23...",1
2,5a7a06935542990198eaf050,4820929,"[26, 650, 9, 327, 27, 1504, 29, 650, 30, 27, 1...","[12285, 276, 12285, 276, 7, 1, 284, 369, 860, ...",0
3,5a7a06935542990198eaf050,1906799,"[26, 650, 9, 327, 27, 1504, 29, 650, 30, 27, 1...","[2371, 650, 2371, 20, 3816, 650, 2, 414, 9, 6,...",0
4,5a879ab05542996e4f30887e,4940122,"[1, 2228827, 65, 7, 77, 2, 6, 1129, 123, 22, 2...","[2228827, 65, 1, 2228827, 65, 7, 20, 249, 233,...",1
5,5a879ab05542996e4f30887e,4097020,"[1, 2228827, 65, 7, 77, 2, 6, 1129, 123, 22, 2...","[1, 2228827, 132, 1, 2228827, 132, 7, 6, 1129,...",1
6,5a879ab05542996e4f30887e,3375253,"[1, 2228827, 65, 7, 77, 2, 6, 1129, 123, 22, 2...","[1, 2228827, 378, 562, 1, 2228827, 5880, 7, 6,...",0
7,5a879ab05542996e4f30887e,1780741,"[1, 2228827, 65, 7, 77, 2, 6, 1129, 123, 22, 2...","[2228827, 2572842, 2228827, 2572842, 7, 6, 876...",0
8,5a8d7341554299441c6b9fe5,3748650,"[1200, 5, 2707208, 227665, 1206617, 748, 6, 15...","[227665, 1206617, 9159, 9067, 227665, 1206617,...",1
9,5a8d7341554299441c6b9fe5,4068311,"[1200, 5, 2707208, 227665, 1206617, 748, 6, 15...","[2011230, 882, 1375109, 2011230, 21614, 882, 1...",1
